In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import matplotlib.pyplot as plt
from botorch.utils.sampling import draw_sobol_samples 
import numpy as np 
import json

import sys
sys.path.append('..')
import sample_utilities.samples as samples
import uuid

import scipy
from scipy.stats.qmc import Sobol
from scipy.stats import qmc

import pandas as pd

## Load bounds

In [17]:
with open('Mesoporous_constants_APS.json', 'r') as f:
    constants = json.load(f)


ctab_lower_vf = constants['ctab']['minimum_mass']/constants['ctab']['stock_concentration_mg_uL']/target_volume  
f127_lower_vf = constants['f127']['minimum_mass']/constants['f127']['stock_concentration_mg_uL']/target_volume

ctab_upper_vf = constants['ctab']['maximum_mass']/constants['ctab']['stock_concentration_mg_uL']/target_volume
f127_upper_vf = constants['f127']['maximum_mass']/constants['f127']['stock_concentration_mg_uL']/target_volume

lower_bounds = [constants['TEOS']['minimum_volume_fraction'],
                constants['ammonia']['minimum_volume_fraction'],
                constants['ethanol']['minimum_volume_fraction'],
                constants['ctab']['minimum_mass'],
                constants['f127']['minimum_mass']]

upper_bounds = [constants['TEOS']['maximum_volume_fraction'],
                constants['ammonia']['maximum_volume_fraction'],
                constants['ethanol']['maximum_volume_fraction'],
                constants['ctab']['maximum_mass'],
                constants['f127']['maximum_mass']]

In [18]:
target_volume = 10000

In [19]:
def manual_mesoporous(sampled_volume_fractions):
    """
    Generates Sobol samples in n dimensions satisfying:
    a) Sum of all components = 1.0
    b) Last component (nth dim) is in [0.4, 1.0]

    Uses rejection sampling: Generates a large pool and filters valid samples.

    Args:
        n (int): Number of dimensions.
        num_samples (int): Number of valid samples to return.
        pool_size (int): Number of initial samples to generate before filtering.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        torch.Tensor: (num_samples, n) tensor of valid samples.
    """

    valid_samples = []


    teos_vol_frac = sampled_volume_fractions[0]
    ammonia_vol_frac = sampled_volume_fractions[1]
    ethanol_vol_frac = sampled_volume_fractions[2]
    # convert surfactant mass to volumes
    ctab_mass = sampled_volume_fractions[3]
    f127_mass = sampled_volume_fractions[4]

    ctab_vol_frac = ctab_mass/constants['ctab']['stock_concentration_mg_uL']/target_volume
    f127_vol_frac = f127_mass/constants['f127']['stock_concentration_mg_uL']/target_volume

    water_from_surfactant_vf = ctab_vol_frac+f127_vol_frac

    

    water_min_vf = constants['water']['minimum_volume_fraction']

    # water that needs to be delivered to meet minimum volume fraction
    water_min_vf_delivered = max(water_min_vf-water_from_surfactant_vf, 0)

    max_other_components = 1 - water_min_vf_delivered

    print(max_other_components)


    sum_nonwater_additions = teos_vol_frac + ammonia_vol_frac + ethanol_vol_frac + ctab_vol_frac + f127_vol_frac
    sum_water_free_additions = teos_vol_frac + ammonia_vol_frac + ethanol_vol_frac
    if sum_nonwater_additions < max_other_components:

        water_vol_frac = 1 - sum_water_free_additions
        sample = [teos_vol_frac, ammonia_vol_frac, ethanol_vol_frac, ctab_mass, f127_mass, water_vol_frac]
        
        print('found valid sample: ', sample)


    return sample

In [32]:
vol_fracs = [0.02, 0.02, 0.25, 35, 130]


In [33]:
baseline_samples = manual_mesoporous(vol_fracs)

1
found valid sample:  [0.02, 0.02, 0.25, 35, 130, 0.71]


In [34]:
uuid_vals = [uuid.uuid4() for val in baseline_samples]

In [35]:
samples_gen = []

row = baseline_samples
sample = samples.MesoporousSample(target_volume=target_volume, reactant_fp='Mesoporous_constants_APS.json', teos_vol_frac=row[0], ammonia_vol_frac=row[1], ethanol_vol_frac=row[2], ctab_mass=row[3], f127_mass=row[4], water_vol_frac=row[5])
sample.calculate_reactant_volumes()
#sample.calculate_silica_mass_concentration()
#sample.calculate_silica_mass_fraction()
#sample.calculate_dilution_volumefraction(target_dilution)
samples_gen.append(sample)
print('final etoh vol: ', sample.ethanol_volume)

TEOS ETOH:  370.0
Solvent etoh total:  370.0
final etoh vol:  2130.0


In [36]:
sample.water_volume

2166.666666666666

In [37]:
uuid_vals = []
teos_volumes = []
ammonia_volumes = []
ethanol_volumes = []
water_volumes = []
ctab_volumes = []
f127_volumes = []
sample_names = []

for i, sample in enumerate(samples_gen):
    sample_names.append(f'SobolBaseline_{i+1}')
    uuid_vals.append(uuid.uuid4())
    teos_volumes.append(sample.teos_volume)
    ammonia_volumes.append(sample.ammonia_volume)
    ethanol_volumes.append(sample.ethanol_volume)
    water_volumes.append(sample.water_volume)
    ctab_volumes.append(sample.ctab_volume)
    f127_volumes.append(sample.f127_volume)\

    total_volume = sample.teos_volume+sample.ammonia_volume + sample.water_volume + sample.ethanol_volume + sample.ctab_volume + sample.f127_volume
    print(f'Total volume sample SobolBaseline_{i+1}: {total_volume}')



Total volume sample SobolBaseline_1: 10000.0


In [38]:
sample_table = pd.DataFrame({'sample_name':sample_names,
                             'uuid':uuid_vals,
                             'teos_volume':teos_volumes,
                             'ammonia_volume':ammonia_volumes,
                             'water_volume':water_volumes,
                             'ethanol_volume':ethanol_volumes,
                             'ctab_volume':ctab_volumes,
                             'F127_volume':f127_volumes
                            })

In [39]:
sample_table

,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,F127_volume
0,SobolBaseline_1,a9415489-89b7-4943-acb9-195a95ad3b0e,570.0,200.0,2166.666667,2130.0,2333.333333,2600.0


In [30]:
vial_volume = 17000
cols = ['teos_volume', 'ammonia_volume', 'ethanol_volume', 'water_volume', 'ctab_volume', 'F127_volume']
for col in cols:
    total_volume = sample_table.iloc[0:24][col].sum()
    n_vials = total_volume/vial_volume
    print(f'Volume for {col}: {total_volume}, vials required: {n_vials}')

Volume for teos_volume: 285.0, vials required: 0.016764705882352942
Volume for ammonia_volume: 100.0, vials required: 0.0058823529411764705
Volume for ethanol_volume: 2315.0, vials required: 0.1361764705882353
Volume for water_volume: 2366.666666666666, vials required: 0.13921568627450978
Volume for ctab_volume: 2333.3333333333335, vials required: 0.13725490196078433
Volume for F127_volume: 2600.0, vials required: 0.15294117647058825


In [31]:
sample_table.to_csv('Mesoporous_LowTEOSAmmonia_testing.csv')